In [1]:
import pandas as pd
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import numpy as np
import scipy

from sklearn.decomposition import LatentDirichletAllocation
from scipy.spatial.distance import cosine
from sklearn.metrics import pairwise_distances
from scipy.sparse import coo_matrix

from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict


In [2]:
df = pd.read_csv('C:/Users/chenq/Desktop/5291ADA/youtube-new/USvideos.csv', index_col=0)

In [3]:
df.columns


Index(['trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [4]:
df.head()

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
video_id,,,,,,,,,,,,,,,
2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [5]:
df['category_id'].nunique()

16

In [6]:
df.shape

(40949, 15)

In [7]:
title = df['title']

In [8]:
stopWord = set(stopwords.words('english'))   

In [9]:
def generate_ngrams(s, n):
    s= s.lower()
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    s = re.sub(r'[0-9]+', ' ', s)
    
    s = nltk.PorterStemmer().stem(nltk.WordNetLemmatizer().lemmatize(s, pos='v'))

    tokens = word_tokenize(s)
    
    tokens = [token for token in tokens if token not in stopWord and len(token) > 3]
    
    output = list(ngrams(tokens, n))
    return(output)


In [10]:
title[0]

'WE WANT TO TALK ABOUT OUR MARRIAGE'

In [11]:
generate_ngrams(title[0], 1)

[('want',), ('talk',), ('marriag',)]

# create df_title with key_word


In [12]:
related_cols = ['title', 'category_id']
df_title = df[related_cols]
df_title['key_word'] = '0'
df_title = df_title.reset_index(drop=True)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
df_title.at[0, 'key_word'] = generate_ngrams(df_title.loc[0, 'title'], n=1)

In [14]:
for i in range(df_title.shape[0]):
    df_title.at[i, 'key_word'] = generate_ngrams(df_title.loc[i, 'title'], n=1)

In [15]:
df_title.head(10)

,title,category_id,key_word
0,WE WANT TO TALK ABOUT OUR MARRIAGE,22,"[(want,), (talk,), (marriag,)]"
1,The Trump Presidency: Last Week Tonight with J...,24,"[(trump,), (presidency,), (last,), (week,), (t..."
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",23,"[(racist,), (superman,), (rudy,), (mancuso,), ..."
3,Nickelback Lyrics: Real or Fake?,24,"[(nickelback,), (lyrics,), (real,), (fake,)]"
4,I Dare You: GOING BALD!?,24,"[(dare,), (going,), (bald,)]"
5,2 Weeks with iPhone X,28,"[(weeks,), (iphone,)]"
6,Roy Moore & Jeff Sessions Cold Open - SNL,24,"[(moore,), (jeff,), (sessions,), (cold,), (ope..."
7,5 Ice Cream Gadgets put to the Test,28,"[(cream,), (gadgets,), (test,)]"
8,The Greatest Showman | Official Trailer 2 [HD]...,1,"[(greatest,), (showman,), (official,), (traile..."
9,Why the rise of the robots won’t mean the end ...,25,"[(rise,), (robots,), (mean,), (work,)]"


In [26]:
df_title_unique = df_title.drop_duplicates(subset = 'title', keep = 'first')
df_title_unique.shape

(6455, 3)

In [27]:
df_title_unique['key_word'].isnull().sum()

0

In [28]:
df_title_unique.loc[0, 'key_word'][0]

('want',)

In [31]:
v = DictVectorizer()
X = v.fit_transform(Counter(f) for f in (df_title_unique['key_word']))
X

<6455x9251 sparse matrix of type '<class 'numpy.float64'>'
	with 32880 stored elements in Compressed Sparse Row format>

In [32]:
X
X.shape[0]

6455

In [33]:
key_word_cols = v.get_feature_names()
len(key_word_cols)

9251

In [34]:
for i in range(len(key_word_cols)):
    s = str(key_word_cols[i])
    s = s[2:-3]
    key_word_cols[i] = s

# LDA


In [35]:
# Run LDA model
lda = LatentDirichletAllocation(n_components=16, max_iter=10, random_state=0)
lda.fit(X)
lda.transform(X)

E:\Anaconda\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


array([[0.015625  , 0.015625  , 0.015625  , ..., 0.015625  , 0.015625  ,
        0.015625  ],
       [0.1328125 , 0.0078125 , 0.0078125 , ..., 0.38281248, 0.0078125 ,
        0.0078125 ],
       [0.0078125 , 0.0078125 , 0.0078125 , ..., 0.0078125 , 0.0078125 ,
        0.0078125 ],
       ...,
       [0.00892857, 0.00892857, 0.00892857, ..., 0.15178571, 0.00892857,
        0.00892857],
       [0.01041667, 0.01041667, 0.01041667, ..., 0.01041667, 0.01041667,
        0.01041667],
       [0.01041667, 0.17708333, 0.01041667, ..., 0.01041667, 0.17708333,
        0.01041667]])

In [36]:
output_df = pd.DataFrame(data = lda.transform(X))
output_df.shape

(6455, 16)

In [37]:
category_id = []
for i in range(output_df.shape[0]):
    prob = output_df.loc[i, :].tolist()
    prob = np.array(prob)
    category = np.argmax(prob)
    category_id.append(category)


In [38]:
output_df['category_id'] = category_id
output_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,category_id
0,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,0.265625,0.515625,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,9
1,0.132813,0.007813,0.007813,0.007813,0.007813,0.007813,0.007813,0.007813,0.007813,0.132813,0.007813,0.257813,0.007813,0.382812,0.007813,0.007813,13
2,0.007813,0.007813,0.007813,0.007813,0.007813,0.882812,0.007813,0.007813,0.007813,0.007813,0.007813,0.007813,0.007813,0.007813,0.007813,0.007813,5
3,0.012500,0.012500,0.012500,0.012500,0.012500,0.012500,0.012500,0.012500,0.012500,0.012500,0.012500,0.812500,0.012500,0.012500,0.012500,0.012500,11
4,0.015625,0.015625,0.015625,0.015625,0.765625,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,0.015625,4


In [39]:
output_df.to_csv('C:/Users/chenq/Desktop/output_df.csv', index=False)

In [40]:
for i in range(0,16):
    print(lda.components_[[i]].size)

9251
9251
9251
9251
9251
9251
9251
9251
9251
9251
9251
9251
9251
9251
9251
9251


In [41]:
# Check the covariance. In each cluster, in oder to check, we run over it with 100000000
for i in range(0,16):
    print(np.cov(lda.components_[[i]])/100000000)

1.9722842124066463e-07
2.7135546935273326e-08
3.318666939208937e-08
3.664028095294841e-08
3.4611563204293166e-08
1.5077910518976917e-08
3.6913824787248573e-08
3.0717533016564197e-08
2.095346015408369e-07
3.856985534495164e-08
2.8307072012470677e-08
5.739856007233624e-08
2.2397556053284266e-08
4.447958789536143e-08
3.778088183151236e-08
2.5896827186111328e-08


In [42]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]])
        print(message)
    print()

In [43]:
print_top_words(lda, key_word_cols, 15)

Topic #0: official, trailer, live, game, world, super, season, teaser, american, week, smith, highlight, awards, jimmy, film
Topic #1: things, tour, refinery, ball, never, google, hart, secret, meet, fair, dead, getting, vanity, fashion, sport
Topic #2: challenge, look, tried, school, cake, giant, everything, baby, kardashian, grace, slow, short, wrong, chocolate, make
Topic #3: first, year, netflix, time, inside, hair, talks, league, justin, disney, show, says, taylor, money, timberlake
Topic #4: trail, lyric, official, feat, honest, wild, trailers, final, room, family, sean, makeover, nintendo, much, speaks
Topic #5: like, ready, special, idol, conan, view, king, coming, charlie, greatest, century, record, trick, dream, santa
Topic #6: james, espn, show, people, battle, lebron, ever, michael, ellen, good, olympics, kitchen, using, stop, cast
Topic #7: full, behind, know, commercial, infinity, avengers, high, today, marvel, performance, highlights, dance, fire, florida, award
Topic #8